# The dojo

## Setup training device

In [6]:
import torch
from torch import device, nn
import torchvision


print(f'PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}')

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Using device: {device}')

PyTorch version: 1.12.1
torchvision version: 0.13.1
Using device: cpu


## Data loading

In [7]:
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=None
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

class_names = train_data.classes

# Turn data into batches (mini-batches)
# More computationally efficent, so we dont store all data in memory
# Gives neural network more changes to update its gradients per epoch
# andrew ng minibatches

BATCH_SIZE = 32

train_dataloader = DataLoader(train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

test_dataloader = DataLoader(test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False)

print(f'Len of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}')
print(f'Len of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}')

Len of train dataloader: 1875 batches of 32
Len of test dataloader: 313 batches of 32


### Check out what's inside the training dataloader

In [8]:
train_features_batch, train_labels_batch = next(iter(train_dataloader))
print(train_features_batch.shape, train_labels_batch.shape)

torch.Size([32, 1, 28, 28]) torch.Size([32])


## Model

### Create a flatten layer - Testing Flatten()

In [9]:
flatten_model = nn.Flatten()

x = train_features_batch[0]
print(f'Shape before flattening: {x.shape}')     # torch.Size([1, 28, 28])

output = flatten_model(x)
print(f'Shape after flattening: {output.shape}') # torch.Size([1, 784])

Shape before flattening: torch.Size([1, 28, 28])
Shape after flattening: torch.Size([1, 784])


### Instantiate model

In [11]:
from model.chrome_vision import ChromeVisionModel

model = ChromeVisionModel(
    input_shape=784,    # 28*28
    hidden_units=10,    # Units in the hidden layer
    output_shape=len(class_names) # one for every class
).to(device)

In [ ]:
from model.chrome_vision import ChromeVisionModelV2

model = ChromeVisionModelV2(
    input_shape=1,      # Color channel: black/white 
    hidden_units=10,    # Units in the hidden layer
    output_shape=(len(class_names)) # Each class
).to(device)

### Setup loss function and optimizer

In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.1)

### Training loop

In [15]:
from timeit import default_timer as timer
from tqdm.auto import tqdm
from model.evaluation import train_step, test_step, accuracy_fn # use torchmetrics.Accuracy()
from model.utilis import print_train_time

torch.manual_seed(42)
train_time_start_on_cpu = timer()

epochs = 3
for epoch in tqdm(range(epochs)):
    print(f'Epoch: {epoch}\n')

    train_step(model=model,
               data_loader=train_dataloader,
               loss_fn=loss_fn,
               optimizer=optimizer,
               accuracy_fn=accuracy_fn,
               device=device)
    
    test_step(model=model,
               data_loader=test_dataloader,
               loss_fn=loss_fn,
               accuracy_fn=accuracy_fn,
               device=device)

# Print time taken
train_time_end_on_cpu = timer()
total_train_time_model = print_train_time(train_time_start_on_cpu, train_time_end_on_cpu, str(next(model.parameters()).device))

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0

Train loss: 0.40594 | Train acc: 85.51%


  0%|          | 0/313 [00:00<?, ?it/s]

Epoch: 1

Train loss: 0.39622 | Train acc: 85.81%


  0%|          | 0/313 [00:00<?, ?it/s]

Epoch: 2

Train loss: 0.39221 | Train acc: 85.99%


  0%|          | 0/313 [00:00<?, ?it/s]

Train time on cpu: 17.427 seconds


### Calculate model results on test dataset

In [17]:
model_results = test_step(model=model,
                           data_loader=test_dataloader,
                           loss_fn=loss_fn,
                           accuracy_fn=accuracy_fn,
                           device=device)

model_results

  0%|          | 0/313 [00:00<?, ?it/s]

{'model_name': 'ChromeVisionModel',
 'model_loss': 0.527795672416687,
 'model_acc': 81.6094249201278}